<a href="https://colab.research.google.com/github/w4bo/teaching-handsondatapipelines/blob/main/materials/02-MachineLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning

Solution of the previous `Hands on`

In [ ]:
!pip install paretoset 
# !pip install auto-sklearn 

from paretoset import paretoset
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
import seaborn as sns

In [ ]:
def preprocess(normalize=True):
    df = pd.read_csv("https://raw.githubusercontent.com/w4bo/handsOnDataPipelines/main/materials/datasets/housing.csv", delimiter=",")

    num_df = df.drop(columns=["ocean_proximity", "median_house_value"])

    # Filling in (i.e., impute) missing values with the median value
    num_df["total_bedrooms"] = num_df["total_bedrooms"].fillna(num_df["total_bedrooms"].median())

    # Add a new column: population_per_household = population / households
    num_df["population_per_household"] = num_df["population"] / num_df["households"]

    # Add a new column: rooms_per_household = total_rooms / households
    num_df["rooms_per_household"] = num_df["total_rooms"] / num_df["households"]

    # Add a new column: bedrooms_per_room = total_bedrooms / total_rooms
    num_df["bedrooms_per_room"] = num_df["total_bedrooms"] / num_df["total_rooms"]

    if normalize:
        # Apply standardization to all the numeric columns
        num_df = (num_df - num_df.mean()) / num_df.std()

    # One hot encode `ocean_proximity` since it is a categorical attribute
    cat_df = pd.get_dummies(df["ocean_proximity"], prefix='ocean_proximity')

    # Join all the dataframes
    # do not change this line
    return pd.concat([num_df, cat_df, df[["median_house_value"]]], axis=1)


df = preprocess()

In [ ]:
tmp = df[["population_per_household", "rooms_per_household", "bedrooms_per_room", "median_house_value"]]
sns.pairplot(tmp.sample(n=1000, random_state=42), markers='+')
plt.show()

Before delving into machine learning, which are the houses the represent a good tradeof between cost and number of rooms?

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1)
df.plot.scatter(x='median_house_value', y='total_bedrooms', ax=ax)

In [ ]:
mask = paretoset(df[["median_house_value", "total_bedrooms"]], sense=["min", "max"])
df[mask].plot.scatter(x='median_house_value', y='total_bedrooms', ax=ax, c='red')
fig

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1)
df.plot.scatter(x="housing_median_age", y="total_bedrooms", c="median_house_value", ax=ax)

In [ ]:
mask = paretoset(df[["housing_median_age", "total_bedrooms", "median_house_value"]], sense=["min", "max", "min"])
df[mask].plot.scatter(x="housing_median_age", y="total_bedrooms", ax=ax, c='red')
fig

Machine Learning is the science (and art) of programming computers so they can learn from data

> Machine Learning is the field of study that gives computers the ability to learn without being explicitly programmed (Arthur Samuel, 1959)

> A computer program is said to learn from experience E with respect to some task T and some performance measure P, if its performance on T, as measured by P, improves with experience E (Tom Mitchell, 1997)


There are many types of Machine Learning algorithms

Classify them in broad categories, based on the following criteria:

- Whether they are trained with human supervision
    - Supervised, unsupervised, semi-supervised, and reinforcement learning
- Whether they can learn incrementally
    - Online, batch learning
- Whether they compare new to known data points, or detect patterns/models in the training
    - Instance-based, model-based learning


**Sklearn**: Machine Learning in Python
- This library is built upon NumPy, SciPy and Matplotlib
    - Open source and commercially usable
- Covers many algorithms
    - Supervised Learning algorithms: Linear Regression, Support Vector Machine, etc.
    - Unsupervised Learning algorithms: clustering, factor analysis, PCA, neural networks, etc.
    - Cross Validation: check the accuracy of supervised models on unseen data
    - Feature extraction: extract the features from data to define the attributes in image and text data


Check also: 
- https://scikit-learn.org/stable/index.html
- https://scikit-learn.org/stable/modules/tree.html

In [ ]:
import sklearn as sk
np.random.seed(42)  # why are we fixing the seed?
print(sk.__version__)

We focus on supervised learning tasks
- The training set you feed to the algorithm includes the desired solutions
- Classification
    - Approximating a mapping function (`f`) from input variables (`X`) to discrete output variables (`y`)
    - The output variables are called labels or categories
    - The mapping function predicts the class or category for a given observation
    - E.g., a spam filter is trained with many example emails along with their class (`spam` or `ham`)
- Regression
    - Approximating a mapping function (`f`) from input variables (`X`) to a continuous output variable (`y`)
    - A continuous output variable is a real-value, such as an integer or floating-point value
    - E.g., predict the price of a car given a set of features (`mileage`, `age`, `brand`, etc.) called predictors


For a supervised learning problem we need:
- Input data along with labels
- Split data between test and training set. How?

Scikit-learn uses data in the form of N-dimensional matrix

- Data as a feature matrix `X` (e.g., a Pandas DataFrame)
    - The samples represent the individual objects described by the dataset (e.g., a `person`)
    - The features describe each sample in a quantitative manner (e.g., `age` and `height`)
- Data as target array `y` (e.g., a Pandas Series)
    - Along with features matrix, we also have the target array (label)


How do we
- distinguish target and feature columns?
- build test and training sets?


In [ ]:
# For illustration only. Sklearn has train_test_split()
def split_train_test(data, test_ratio):
    # get a list of random numbers
    shuffled_indices = np.random.permutation(len(data))
    # define the size of test dataset
    test_set_size = int(len(data) * test_ratio)
    # the test dataset includes some indexes
    test_indices = shuffled_indices[:test_set_size]
    # the train dataset includes the others (no overlapping)
    train_indices = shuffled_indices[test_set_size:]
    # return the two dataframes
    return data.iloc[train_indices], data.iloc[test_indices]


# get the train and test dataframes
df_train, df_test = split_train_test(df, 0.2)

# get the train feature matrix
X_train = df_train.drop(columns=["median_house_value"]).to_numpy()
# get the test feature matrix
X_test = df_test.drop(columns=["median_house_value"]).to_numpy()
# get the train label array
y_train = df_train["median_house_value"].to_numpy()
y_test = df_test["median_house_value"].to_numpy()  # get the test label array

In [ ]:
from sklearn.model_selection import train_test_split
y = df["median_house_value"]  # labels
X = df.drop(columns=["median_house_value"])  # input data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
X_train

In [ ]:
y_train

Estimator

- A consistent interface for a wide range of ML applications
- The algorithm that learns from the data (fitting the data) is an estimator
- It can be used with any of the algorithms like classification, regression, and clustering

All the parameters can be set when creating the estimator

    estimator = Estimator(param1=1, param2=2)
    estimator.param1
All estimator objects expose a fit method that takes a dataset

    estimator.fit(X)
Once data is fitted with an estimator, all the estimated parameters will be the attributes of the estimator object ending by an underscore

    estimator.estimated_param_

Sklearn in action

1. Choose a class of model
    - Import the appropriate Estimator class from Scikit-learn (e.g., a decision tree)
1. Choose model hyperparameters
1. Model Fitting
    - Fit the model by calling `fit()` method of the model instance
1. Applying the model to new data
    - For supervised learning, use `predict()` method to predict the labels for unknown data. 
    - For unsupervised learning, use `predict()` or `transform()` to infer properties of the data.


## Regression

In [ ]:
# Choose a class of model, Apply linear regression
# (https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)
from sklearn.linear_model import LinearRegression  # import the model
# Choose model hyperparameters and initialize the model (i.e., the estimator)
lin_reg = LinearRegression(fit_intercept=True)
# Model fitting
lin_reg.fit(X_train, y_train)  # train it
# "LinearRegression fits a linear model with coefficients w = (w1, ..., wp) to minimize the residual
# sum of squares between the observed targets in the dataset, and the targets predicted by the linear approximation."
lin_reg.coef_

![image](https://user-images.githubusercontent.com/18005592/232785531-1258fd6b-78a8-4f0a-9bde-bcc7f416f14f.png)

In [ ]:
# predict the cost of houses in the test set
housing_predictions = lin_reg.predict(X_test)
housing_predictions

This checklist can help you while building your projects

Frame the problem and look at the big picture

- ✔ Define the objective in business terms
- ✖ How should performance be measured? (let's do this!)


We are facing a regression problem

- A typical performance measure for regression problems is the Root Mean Square Error (RMSE)
- RMSE is the standard deviation of the residuals (prediction errors)
- Residuals are a measure of how far from the regression line data points are; RMSE is a measure of how spread out these residuals are


![image](https://user-images.githubusercontent.com/18005592/232786035-abaca941-12c5-44b0-9085-e9908512803d.png)

In [ ]:
lin_mse = mean_squared_error(y_test, housing_predictions)  # check the error
lin_rmse = np.sqrt(lin_mse)  # apply the squared root of the error
print(lin_rmse)  # print it

# visualize some predictions
df = pd.DataFrame({'y_test': y_test[:10].to_numpy(), 'y_pred': housing_predictions[:10]}, index=[x for x in range(10)])
df.plot.bar(rot=0)


This is better than nothing, but clearly not a great score: most districts’ `median_housing_values` range between 120K USD and  265K USD, so a typical prediction error of ~70K USD is not very satisfying. This is an example of a model underfitting the training
data.

https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html

"A random forest is a meta estimator that fits a number of classifying decision trees on various sub-samples of the dataset and uses averaging to improve the predictive accuracy and control over-fitting. The sub-sample size is controlled with the max_samples parameter if bootstrap=True (default), otherwise the whole dataset is used to build each tree."

In [ ]:
from sklearn.ensemble import RandomForestRegressor  # import the model


def run_forest(n_estimators, max_features, min_samples_split=5):
    # initialize the model (i.e., the estimator)
    forest_reg = RandomForestRegressor(n_estimators=n_estimators, max_features=max_features, random_state=42)
    forest_reg.fit(X_train, y_train)  # train it
    # predict the cost of houses in the test set
    housing_predictions = forest_reg.predict(X_test)

    forest_rmse = mean_squared_error(y_test, housing_predictions, squared=True)  # check the error
    forest_rmse = np.sqrt(forest_rmse)  # apply the squared root of the error
    print(forest_rmse)

    df = pd.DataFrame({'y_test': y_test[:10].to_numpy(
    ), 'y_pred': housing_predictions[:10]}, index=[x for x in range(10)])
    df.plot.bar(rot=0)
    return forest_reg

In [ ]:
forest_reg = run_forest(100, 1.0)

Look at parameters used by our current forest

In [ ]:
forest_reg.get_params()

## Hyperparameter tuning

Hyper-parameters: parameters that are not directly learnt within estimators
- In scikit-learn they are passed as arguments to the constructor of the estimator classes
- Any parameter provided when constructing an estimator may be optimized

        estimator.get_params()
A search consists of:
- an estimator
- a parameter space
- a method for searching or sampling candidates
- a cross-validation scheme
- a score function

### Cross validation

![image](https://user-images.githubusercontent.com/18005592/232793622-e98002b4-d58b-465b-8a3d-e7b3d0dafe6f.png)

![Picture1](https://user-images.githubusercontent.com/18005592/232802005-d3a1aff6-23d8-4704-8a3f-a219d2155d30.png)

There are many search algorithms:

- *Random*
    - "The parameters of the estimator used to apply these methods are optimized by cross-validated search over parameter settings. In contrast to GridSearchCV, not all parameter values are tried out, but rather a fixed number of parameter settings is sampled from the specified distributions. The number of parameter settings that are tried is given by n_iter."
    - https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html
- Grid
- AutoML



In [ ]:
from sklearn.model_selection import RandomizedSearchCV

random_grid = {
    # Number of trees in random forest
    'n_estimators': [10, 25, 50, 100, 150],
    # Number of features to consider at every split
    'max_features': [1.0, 'sqrt'],
    'min_samples_split': [2, 5]
}

# Fit the random search model
rf_random = RandomizedSearchCV(estimator=forest_reg, param_distributions=random_grid, cv=3, verbose=2, random_state=42)
rf_random.fit(X_train, y_train)
rf_random.best_params_


In [ ]:
run_forest(n_estimators=100, min_samples_split=5, max_features='sqrt')

#### Dimensionality reduction

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
result = pca.fit_transform(X_train.values)

plt.scatter(
    x=result[:, 0],
    y=result[:, 1],
    c=y_train,
    cmap='viridis'
)

In [ ]:
# Dump components relations with features
pd.DataFrame(pca.components_, columns=X_train.columns, index=['PC1', 'PC2'])

In [ ]:
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2)
result = tsne.fit_transform(X_train.head(2000))

plt.scatter(
    x=result[:, 0],
    y=result[:, 1],
    c=y_train.head(2000).apply(lambda x: 0 if x == 'low' else (1 if x == 'medium' else 2)),
    cmap='viridis'
)

## Classification

Can we turn this into a classification problem?

In [ ]:
df = preprocess(False)  # clean the dataset withtout standardization
bins = ["low", "medium", "high"]  # the labels we are gonna use
# map bins of values into labels
df["median_house_value"] = pd.cut(df["median_house_value"], 3, labels=bins)
df["median_house_value"].hist()  # plot them


For the sake of performance, we are going to use a manually defined set of features

In [ ]:
y = df["median_house_value"]  # labels
# consider only some features from the train dataset
X = df[["latitude", "longitude", "median_income", "population", "households", "total_bedrooms"]]
# split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

Computing performance


![Picture2](https://user-images.githubusercontent.com/18005592/232803278-7a0e5182-4267-410a-a809-7edc54a47980.png)

![Picture3](https://user-images.githubusercontent.com/18005592/232803261-1cae7aac-e3fc-4467-bb45-65455d7a34a7.png)

- Precision: $\frac{tp}{tp + fp}$
- Recall: $\frac{tp}{tp + fn}$
- Accuracy: $\frac{tp + tn}{tp + tn + fp + fpn}$
- F1-score: $2 \cdot \frac{precision \cdot recall}{precision + recall}$

Summing up 
- Accuracy is used when TP and TN are more important while F1-score is used when FN and FP are
- Accuracy can be used when the class distribution is similar, while F1-score is a better when there are imbalanced classes


In [ ]:
from sklearn.metrics import accuracy_score
from sklearn import tree
# initialize the model
clf = tree.DecisionTreeClassifier(criterion="entropy", random_state=42)
clf = clf.fit(X_train, y_train)  # train it
# predict the cost of houses in the test set
housing_predictions = clf.predict(X_test)
# check the error
accuracy = accuracy_score(y_test, housing_predictions)
accuracy

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, housing_predictions, target_names=bins))

In [ ]:
plt.figure(figsize=(20,12))
tree.plot_tree(clf, fontsize=10, feature_names=list(X_train.columns), class_names=bins, max_depth=4)
plt.show()

In [ ]:
from sklearn.ensemble import RandomForestClassifier

forest_cls = RandomForestClassifier(criterion="entropy", random_state=42) # initialize the model
forest_cls = forest_cls.fit(X_train, y_train) # train it
housing_predictions = forest_cls.predict(X_test) # predict the cost of houses in the test set

accuracy = accuracy_score(y_test, housing_predictions) # check the error
accuracy

In [ ]:
forest_cls.get_params()

In [ ]:
random_grid = {'n_estimators': [50, 100, 150], # Number of trees in random forest
               'max_features': [1.0, 'sqrt'], # Number of features to consider at every split
               'min_samples_leaf': [1, 5],
               'max_depth': [2, 5]
              }

# Fit the random search model
rf_random = RandomizedSearchCV(estimator=forest_cls, param_distributions=random_grid, cv=3, verbose=2, random_state=42)
rf_random.fit(X_train, y_train)
rf_random.best_params_

In [ ]:
forest_cls = RandomForestClassifier(criterion="gini", random_state=42, n_estimators=50, max_features="sqrt", min_samples_leaf=1, max_depth=5) 
forest_cls = forest_cls.fit(X_train, y_train) # train it
housing_predictions = forest_cls.predict(X_test) # predict the cost of houses in the test set

accuracy = accuracy_score(y_test, housing_predictions) # check the error
accuracy

In [ ]:
forest_cls.get_params()

## AutoML with auto-sklearn

In [ ]:
# scale_mapper = {"low": 1, "medium": 2, "high": 3}
# y_train = y_train.replace(scale_mapper)
# y_test = y_test.replace(scale_mapper)
# y_train

In [ ]:
# import autosklearn.regression
# import autosklearn.classification
# import autosklearn.metrics

# # Create and train the estimator 
# # estimator_askl = autosklearn.regression.AutoSklearnRegressor(
# estimator_askl = autosklearn.classification.AutoSklearnClassifier(
#     time_left_for_this_task=180,
#     seed=42, 
#     resampling_strategy='cv',
#     resampling_strategy_arguments={'folds': 3},
#     n_jobs=2,
#     metric=autosklearn.metrics.accuracy
#     # metric=autosklearn.metrics.root_mean_squared_error,
# )
# # Auto-sklearn ingests the pandas dataframe and detects column types
# estimator_askl.fit(X_train, y_train)

In [ ]:
# print("Available CLASSIFICATION metrics autosklearn.metrics.*:")
# print("\t*" + "\n\t*".join(autosklearn.metrics.CLASSIFICATION_METRICS))
# print("Available REGRESSION autosklearn.metrics.*:")
# print("\t*" + "\n\t*".join(autosklearn.metrics.REGRESSION_METRICS))

In [ ]:
# Score the model
# prediction = estimator_askl.predict(X_test)
# prediction

In [ ]:
# performance_askl = autosklearn.metrics.accuracy(y_test, prediction)
# performance_askl = autosklearn.metrics.mean_squared_error(y_test, prediction, squared=False)
# print(f"Auto-Sklearn Classifier performance is {performance_askl}")

In [ ]:
# print(estimator_askl.show_models())